In [2]:
#load both training and testing data
import pandas as pd 
train_data=pd.read_csv(r"E:\Projects\Victor-Files\Digit_recognizer\data\train.csv")
test_data=pd.read_csv(r"E:\Projects\Victor-Files\Digit_recognizer\data\test.csv")

In [3]:
train_data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [4]:
train_data.iloc[[0,1], 0]

0    1
1    0
Name: label, dtype: int64

In [5]:
train_data.loc[[0,1], 'label']

0    1
1    0
Name: label, dtype: int64

In [6]:
train_data.loc[0:2, 'label']

0    1
1    0
2    1
Name: label, dtype: int64

In [7]:
train_data.iloc[0:2,0]

0    1
1    0
Name: label, dtype: int64

In [8]:
filt_7=train_data['label'] == 3
train_data.loc[filt_7,:]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41971,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41974,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41984,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41985,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
filt = (train_data['label'] == 3) | (train_data['label'] == 7)
filtered_data = train_data.loc[filt, :]
filtered_data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41983,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41984,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41985,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41990,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
